In [39]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()

# Récupérer les variables d'environnement
server = os.getenv("DB_SERVER")
database = os.getenv("DB_NAME")
username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
driver = os.getenv("DB_DRIVER").replace("{", "").replace("}", "")

# Chaîne de connexion
connection_string = f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
DATABASE_URL = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver.replace(' ', '+')}"

# Tester la connexion avec pyodbc
try:
    conn = pyodbc.connect(connection_string)
    print("Connexion réussie avec pyodbc!")
    conn.close()
except pyodbc.Error as ex:
    print("Erreur lors de la connexion avec pyodbc:", ex)

# Créer le moteur SQLAlchemy
engine = create_engine(DATABASE_URL)


Connexion réussie avec pyodbc!


In [40]:
# Charger les tables nécessaires
try:
    movies = pd.read_sql("SELECT * FROM appmovieschema.Movie_Table", engine)
    genres = pd.read_sql("SELECT * FROM appmovieschema.Genre_Table", engine)
    movie_genres = pd.read_sql("SELECT * FROM appmovieschema.MovieGenre_Table", engine)

    # Afficher un aperçu des données
    print("Movies Table:")
    print(movies.head())
    
    print("\nGenres Table:")
    print(genres.head())
    
    print("\nMovie Genres Table:")
    print(movie_genres.head())
except Exception as e:
    print(f"Erreur lors du chargement des données : {e}")


INFO:sqlalchemy.engine.Engine:SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:SELECT schema_name()
INFO:sqlalchemy.engine.Engine:[generated in 0.00074s] ()
INFO:sqlalchemy.engine.Engine:SELECT CAST('test max support' AS NVARCHAR(max))
INFO:sqlalchemy.engine.Engine:[generated in 0.00081s] ()
INFO:sqlalchemy.engine.Engine:SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
INFO:sqlalchemy.engine.Engine:[generated in 0.00069s] ()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] 

Movies Table:
   movie_id                         title release_date  \
0         1  Aquaman and the Lost Kingdom   2023-12-20   
1         2                         Wonka   2023-12-06   
2         3                          Wish   2023-11-13   
3         4                     Migration   2023-12-06   
4         5                   The Marvels   2023-11-08   

                                         poster_link  
0  https://image.tmdb.org/t/p/original/7lTnXOy0iN...  
1  https://image.tmdb.org/t/p/original/qhb1qOilap...  
2  https://image.tmdb.org/t/p/original/AcoVfiv1rr...  
3  https://image.tmdb.org/t/p/original/ldfCF9RhR4...  
4  https://image.tmdb.org/t/p/original/9GBhzXMFjg...  

Genres Table:
   genre_id       genre_name
0         1           Action
1         2      Documentary
2         3  Science Fiction
3         4          Mystery
4         5         Thriller

Movie Genres Table:
   movie_id  genre_id
0         1         1
1         1         8
2         1         9
3        

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Créer une description des films à partir des genres
movies = movies.merge(movie_genres.groupby('movie_id')['genre_id'].apply(lambda x: ' '.join(map(str, x))).reset_index(), on='movie_id', how='left')
movies.rename(columns={'genre_id': 'genres'}, inplace=True)

# Remplacer les NaN par des chaînes vides
movies['genres'] = movies['genres'].fillna('')

# Utiliser TfidfVectorizer pour transformer les genres en vecteurs
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres'])

# Calculer la similarité cosinus entre les films
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [42]:
# Fonction de recommandation basée sur la similarité des genres
def get_recommendations(favorite_movie_titles, cosine_sim=cosine_sim, top_n=10):
    # Obtenir les indices des films correspondant aux titres
    favorite_indices = [movies[movies['title'] == title].index[0] for title in favorite_movie_titles]
    
    # Obtenir les scores de similarité pour ces films avec tous les autres films
    sim_scores = cosine_sim[favorite_indices].mean(axis=0)
    
    # Trier les films en fonction des scores de similarité
    sim_scores = list(enumerate(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtenir les indices des films les plus similaires
    movie_indices = [i[0] for i in sim_scores if i[0] not in favorite_indices][:top_n]
    
    # Retourner les films les plus similaires
    return movies.iloc[movie_indices]

# Exemple d'utilisation avec des titres de films préférés fictifs
favorite_movie_titles = ["The Holdovers", "The Godfather"]  # Remplacez par des titres existants dans votre base de données
recommended_movies = get_recommendations(favorite_movie_titles)
print(f"Recommended Movies for {favorite_movie_titles}:")
print(recommended_movies[['title', 'genres']])


Recommended Movies for ['The Holdovers', 'The Godfather']:
                                                title     genres
5                                       Sixty Minutes       1 14
7                                             Radical         14
9                                                Lift  1 15 7 14
12                                      The Beekeeper     1 5 14
20                                    Badland Hunters     1 3 14
31  The Hunger Games: The Ballad of Songbirds & Sn...     14 3 1
44             Rebel Moon - Part One: A Child of Fire     3 14 1
52                                   Sound of Freedom       1 14
57                    El padrino: The Latin Godfather     1 7 14
59                                               Acid       14 9


In [43]:
# Afficher les genres des films préférés
print("Genres des films préférés :")
print(movies[movies['title'].isin(favorite_movie_titles)][['title', 'genres']])

# Afficher les genres des films recommandés
print("Genres des films recommandés :")
print(recommended_movies[['title', 'genres']])


Genres des films préférés :
             title genres
167  The Godfather   14 7
199  The Holdovers  15 14
Genres des films recommandés :
                                                title     genres
5                                       Sixty Minutes       1 14
7                                             Radical         14
9                                                Lift  1 15 7 14
12                                      The Beekeeper     1 5 14
20                                    Badland Hunters     1 3 14
31  The Hunger Games: The Ballad of Songbirds & Sn...     14 3 1
44             Rebel Moon - Part One: A Child of Fire     3 14 1
52                                   Sound of Freedom       1 14
57                    El padrino: The Latin Godfather     1 7 14
59                                               Acid       14 9


In [44]:
import pickle

# Sauvegarder le modèle et les données nécessaires dans un fichier .pkl
model_data = {
    "movies": movies,
    "cosine_sim": cosine_sim
}

with open("movie_recommendation_model.pkl", "wb") as f:
    pickle.dump(model_data, f)

print("Modèle sauvegardé avec succès!")


Modèle sauvegardé avec succès!
